# Reimagined-Vis

# v_a11y_lint

In [15]:
import altair as alt
from vega_datasets import data
import v_a11ylint as val
import test_fns as tf

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Note: we are currently only linting for the following:
 * Existence of title
 * Length of title (is it descriptive?)
 * Color shemes (is a color scale too dissimilar, using CIEDE2000 distance formula for 2 x "Just Noticeable Difference")
 * Text to background color (contrast should be above 4.5:1 as defined by WCAG guidelines)
 * Font sizes (all must be more than 1.2 EM (16 pt) as defined by WCAG guidelines

Aditionally, the Altair visualization library allows users to set a global theme for all charts created after the theme is set as well as chart-specific attributes. Currently, we are only linting the entire theme. Therefore, issues will be printed which may not be relevant to the type of chart plotted, e.g. "legend font too small" printed when chart does not have nor need a legend. Our next step for our final prototype is to have only relevant error messages print.

## Example One: Not very accesibile visualization

### First, we set our theme

In [83]:
tf.bad_theme()
tf.set_theme(tf.bad_theme)

### Then, we make our chart

In [84]:
basic_bar = tf.make_basic_bar()
basic_bar

alt.Chart(...)

### Now, we run our lint software on our chart.

In [85]:
val.run_lint(basic_bar)

{'color': {'text to background': 'colors too similar',
  'title color to background': 'colors too similar'},
 'font': {'axisX': {'titleFontSize': 'font size too small: font should be at least 16px'},
  'axisY': {'titleFontSize': 'font size too small: font should be at least 16px'},
  'text': {'fontSize': 'font size too small: font should be at least 16px'},
  'legend': {'titleFontSize': 'font size too small: font should be at least 16px'}},
 'title': 'Chart needs title'}

As we can see above, this chart has a lot of issues! In the cells below, we update our theme (source code for each theme can be found out github [here](https://github.com/katykoenig/reimagined-vis/blob/master/v_a11y_lint/test_fns.py).

## Example Two: A Better Visualization

### Now, we set our updated theme.

In [86]:
tf.better_theme()
tf.set_theme(tf.better_theme)

### Here's our updated chart.

In [87]:
better_chart = tf.make_basic_bar('A vs. B')
better_chart

alt.Chart(...)

### Again, we run our lint software on this chart

In [88]:
val.run_lint(better_chart)

{'color': {'text to background': 'colors too similar'},
 'font': {'legend': {'titleFontSize': 'font size too small: font should be at least 16px'},
  'title length': 'Chart title lacks description'}}

And look, we have fewer issues than in our original chart. Let's address these below!

## Example Three: An Accessible Visualization

### Update our theme

In [89]:
tf.best_theme()
tf.set_theme(tf.best_theme)

### Make our chart

In [90]:
best_chart = tf.make_basic_bar('Variable A by Variable B: Category D Highest')
best_chart

alt.Chart(...)

### Run our v_a11y_lint

In [91]:
val.run_lint(best_chart)

'Visualization is Accessible!'

Yay! Our chart is now much more accessible for low vision users.

## Your turn

Below, we offer the chance for users to set/edit their own theme, quickly create a chart of their choosing from the Altair examples library, and then run v_a11y_lint on this chart to check for accessible. Users can then update the them to make the visualization more accessible.

### Set a theme

In [80]:
# credit to Lilian Huang for this theme
# https://lilianhj.github.io/dataviz-static-portfolio/

# build/edit for your own theme
def class_theme():
    main_palette = ["#385ed4","#55b748","#db2b27","#b589da",
                    "#b75f31","#1696d2","#fdbf11","#ff1ae4"]
    sequential_palette = ["#cfe8f3","#aaecff","#a2d4ec",
                          "#73bfe2","#46abdb","#1696d2","#12719e",
                          "#0a4c6a","#062635"]
    return {
    "config": {
            "title": {
                "font": "Futura",
                "fontSize": 18,
                "anchor": "middle",
                "color": "darkblue",
            },
         "axisX": {
                "grid": False,
                "tickSize": 6,
             "labelFontSize": 10,
             "titleFontSize": 12,
         },
        "axisY": {
                "labelFontSize": 10,
                "tickSize": 6,
            "titleFontSize": 12,
         },
        "background": "white",
        "text": {
               "color": "#686863",
               "fontSize": 10,
               "fontWeight": 400,
            "baseline": "top",
            "filled": True,
            "lineBreak": "\n",
           },
            "bar": {
                "fill": "#1696d2"
            },
            "line": {
               "strokeWidth": 3,
           },
        "range": {
                "category": main_palette,
                "diverging": "blueorange",
            "ramp": sequential_palette,
            "ordinal": sequential_palette,
            "heatmap": sequential_palette
            },
        "legend": {
                "titleFontSize": 12
            }
    },
    }

alt.themes.register("my_custom_theme", class_theme)
alt.themes.enable("my_custom_theme")

ThemeRegistry.enable('my_custom_theme')

### Now, make a chart!
Find examples here: https://altair-viz.github.io/gallery/index.html

In [48]:
source = data.movies.url
# Top 10 movies by IMBD rating
test_chart = alt.Chart(
    source,
).mark_bar().encode(
    x=alt.X('Title:N', sort='-y'),
    y=alt.Y('IMDB_Rating:Q'),
    color=alt.Color('IMDB_Rating:Q')

).transform_window(
    rank='rank(IMDB_Rating)',
    sort=[alt.SortField('IMDB_Rating', order='descending')]
).transform_filter(
    (alt.datum.rank < 10)
)

test_chart

alt.Chart(...)

In [49]:
val.run_lint(test_chart)

{'color': {'text to background': 'colors too similar'},
 'font': {'axisX': {'titleFontSize': 'font size too small: font should be at least 16px'},
  'axisY': {'titleFontSize': 'font size too small: font should be at least 16px'},
  'text': {'fontSize': 'font size too small: font should be at least 16px'},
  'legend': {'titleFontSize': 'font size too small: font should be at least 16px'}},
 'title': 'Chart needs title'}

In [50]:
source = data.iowa_electricity()

test2 = alt.Chart(source).mark_area().encode(
    x="year:T",
    y="net_generation:Q",
    color="source:N"
)

test2

alt.Chart(...)

In [51]:
val.run_lint(test2)

{'color': {'text to background': 'colors too similar'},
 'font': {'axisX': {'titleFontSize': 'font size too small: font should be at least 16px'},
  'axisY': {'titleFontSize': 'font size too small: font should be at least 16px'},
  'text': {'fontSize': 'font size too small: font should be at least 16px'},
  'legend': {'titleFontSize': 'font size too small: font should be at least 16px'}},
 'title': 'Chart needs title'}

In [10]:
def class_theme_updated():
    main_palette = ["#385ed4","#55b748","#db2b27","#b589da",
                    "#b75f31","#1696d2","#fdbf11","#ff1ae4"]
    sequential_palette = ["#cfe8f3","#aaecff","#a2d4ec",
                          "#73bfe2","#46abdb","#1696d2","#12719e",
                          "#0a4c6a","#062635"]
    return {
    "config": {
            "title": {
                "font": "Futura",
                "fontSize": 18,
                "anchor": "middle",
                "color": "darkblue",
            },
         "axisX": {
                "grid": False,
                "tickSize": 6,
             "labelFontSize": 16,
             "titleFontSize": 16,
         },
        "axisY": {
                "labelFontSize": 16,
                "tickSize": 6,
            "titleFontSize": 16,
         },
        "background": "white",
        "text": {
               "color": "#686863",
               "fontSize": 16,
               "fontWeight": 400,
            "baseline": "top",
            "filled": True,
            "lineBreak": "\n",
           },
            "bar": {
                "fill": "#1696d2"
            },
            "line": {
               "strokeWidth": 3,
           },
        "range": {
                "category": main_palette,
                "diverging": "blueorange",
            "ramp": sequential_palette,
            "ordinal": sequential_palette,
            "heatmap": sequential_palette
            },
        "legend": {
                "titleFontSize": 16
            }
    },
    }

alt.themes.register("my_custom_theme", class_theme_updated)
alt.themes.enable("my_custom_theme")

ThemeRegistry.enable('my_custom_theme')

In [11]:
source = data.iowa_electricity()
test3 = alt.Chart(source).mark_area().encode(
    x="year:T",
    y="net_generation:Q",
    color="source:N"
)

test3

alt.Chart(...)

In [12]:
val.run_lint(test3)

{'color': {'text to background': 'colors too similar'},
 'title': 'Chart needs title'}

And now, you've made a more accessible visualization without having it be a "special" visualization/tool for those with low vision.